<a href="https://colab.research.google.com/github/HaoyuanDu12854396/ML2019/blob/master/Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Assignment 2: Practical Mechine Learning Project


##Define Problem
I tried to alternatively implement a mechanical learning algorithm, but I don't have a good idea to do that, so I finally decided to implement a simple data mining algorithm. I will talk about id3 algorithm. In this article, I introduce the id3 algorithm and show how computer program code corresponds to algorithm steps. In this report, I will use a dummy data which is used for learning data analysis to supplement the description, and this data is divided into two parts, one part is Training and the other part is Testing, in my work, they will be integrated and processed. And the conclusions drawn from this data are not special and can be replicated.
These are an overview of the data and an overview of the data after they were integrated for discretization and normalizetion.(I deal with it by R)

In [0]:
# activate R magic
%load_ext rpy2.ipython

In [2]:
#Get data from my github
!git clone https://github.com/HaoyuanDu12854396/ML2019/

Cloning into 'ML2019'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 99 (delta 5), reused 0 (delta 0), pack-reused 84
Unpacking objects: 100% (99/99), done.


In [3]:
!pwd
!ls -R

/content
.:
ML2019	sample_data

./ML2019:
 Assignment_1.ipynb	    Input.csv	  Training.csv
'Decision tree model.png'   Testing.csv   Work.csv

./sample_data:
anscombe.json		      mnist_test.csv
california_housing_test.csv   mnist_train_small.csv
california_housing_train.csv  README.md


In [0]:
%%R
#Importing csv files
Testing <- read.csv('./ML2019/Testing.csv')
Training <- read.csv('./ML2019/Training.csv')
Input <- read.csv('./ML2019/Input.csv')

These are a summary of the data and a summary of the data after they were integrated for discretization and normalizetion.

Summary of Testing;

In [5]:
%%R
#Summary of Testing
summary(is.na(Testing))

   row.ID           age             job           marital       
 Mode :logical   Mode :logical   Mode :logical   Mode :logical  
 FALSE:8238      FALSE:8238      FALSE:8238      FALSE:8238     
 education        default         housing           loan        
 Mode :logical   Mode :logical   Mode :logical   Mode :logical  
 FALSE:8238      FALSE:8238      FALSE:8238      FALSE:8238     
  contact          month         day_of_week      duration      
 Mode :logical   Mode :logical   Mode :logical   Mode :logical  
 FALSE:8238      FALSE:8238      FALSE:8238      FALSE:8238     
  campaign         pdays          previous        poutcome      
 Mode :logical   Mode :logical   Mode :logical   Mode :logical  
 FALSE:8238      FALSE:8238      FALSE:8238      FALSE:8238     
 emp.var.rate    cons.price.idx  cons.conf.idx   euribor3m      
 Mode :logical   Mode :logical   Mode :logical   Mode :logical  
 FALSE:8238      FALSE:8238      FALSE:8238      FALSE:8238     
 nr.employed    
 Mode :l

Summary of Training;

In [6]:
%%R
#Summary of Training
summary(is.na(Training))

   row.ID           age             job           marital       
 Mode :logical   Mode :logical   Mode :logical   Mode :logical  
 FALSE:32950     FALSE:32950     FALSE:32950     FALSE:32950    
 education        default         housing           loan        
 Mode :logical   Mode :logical   Mode :logical   Mode :logical  
 FALSE:32950     FALSE:32950     FALSE:32950     FALSE:32950    
  contact          month         day_of_week      duration      
 Mode :logical   Mode :logical   Mode :logical   Mode :logical  
 FALSE:32950     FALSE:32950     FALSE:32950     FALSE:32950    
  campaign         pdays          previous        poutcome      
 Mode :logical   Mode :logical   Mode :logical   Mode :logical  
 FALSE:32950     FALSE:32950     FALSE:32950     FALSE:32950    
 emp.var.rate    cons.price.idx  cons.conf.idx   euribor3m      
 Mode :logical   Mode :logical   Mode :logical   Mode :logical  
 FALSE:32950     FALSE:32950     FALSE:32950     FALSE:32950    
 nr.employed      Final_Y

Summary of the data after they were integrated for discretization and normalizetion:

In [7]:
%%R
#Summary of the data after they were integrated for discretization and normalizetion
 #normalize input data 
  normalize <- function(x) { return ((x-min(x))/(max(x)-min(x))) }
  input <- as.data.frame(lapply(Input[1:(length(Input[1,])-1)],normalize)) 
  print(str(input))

'data.frame':	41188 obs. of  19 variables:
 $ age           : num  0.481 0.494 0.284 0.481 0.346 ...
 $ job           : num  0.273 0.636 0 0.636 0.636 ...
 $ marital       : num  0.333 0.333 0.333 0.333 0.333 ...
 $ education     : num  0 0.429 0.143 0.429 0.286 ...
 $ housing       : num  0 0 0 0 0 0 0 1 1 0 ...
 $ loan          : num  0 0 0 1 0 0 0 0 0 0 ...
 $ contact       : num  1 1 1 1 1 1 1 1 1 1 ...
 $ month         : num  0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 ...
 $ day_of_week   : num  0 0 0 0 0 0 0 0 0 0 ...
 $ duration      : num  0.0531 0.0303 0.0307 0.0624 0.0403 ...
 $ campaign      : num  0 0 0 0 0 0 0 0 0 0 ...
 $ pdays         : num  1 1 1 1 1 1 1 1 1 1 ...
 $ previous      : num  0 0 0 0 0 0 0 0 0 0 ...
 $ poutcome      : num  0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 ...
 $ emp.var.rate  : num  0.938 0.938 0.938 0.938 0.938 ...
 $ cons.price.idx: num  0.699 0.699 0.699 0.699 0.699 ...
 $ cons.conf.idx : num  0.603 0.603 0.603 0.603 0.603 ...
 $ euribor3m     : num  

##Algorithm Discussion
ID3 is a classic algorithm in machine learning. The decision tree is a tree structure similar to the flow chart : each internal node represents a test on an attribute, each A branch represents an attribute output, and each leaf node represents a class or class distribution, and the topmost layer of the tree is a root node. 



###ID3 algorithm step description
The overall idea: each time the sample is divided according to the best partitioning attribute, and the category with the largest number of samples in the leaf node is the category of the current leaf node.
The following picture is the decision tree I exported via R
![替代文字](https://raw.githubusercontent.com/HaoyuanDu12854396/ML2019/master/Decision%20tree%20model.png)

Input: Data Set（Any preprocessed data set,like Work.csv whose difference for Input.csv is that I deleted the title and add a row id.）

Output: Decision tree

####1. Input

In [0]:
import pandas
import numpy as np
columna = ["age",	"job","marital","education","housing",
           "loan","contact","month","day_of_week","duration",
           "campaign","pdays","previous","poutcome","emp.var.rate","cons.price.idx"
           "cons.conf.idx",	"euribor3m","nr.employed","Final_Y"]
Work = pandas.read_csv('./ML2019/Work.csv', names=columna)
change_data = np.array(Work)
data=change_data.tolist()
Work=list(Work.columns)


####2. Definition entropy

In [0]:
from math import log
def calc_entropy(Work):
  num=lan(Work)#statistical sample number
  labelcouts={}#initialize label
#build label
  for feature in Work:
    ylabel=feature[-1]
    if ylable not in labelcounts.keys():
      labelcounts[ylabel]=0 #add label
    labelcunts[ylabel]+=1 
  entropy=0.0#initialize entropy
#calculat entropy
  for key in labelcounts:
    prob=float(labelcounts[key])/num#calculat probability
    entropy -=prob*log(prob,2)#calculat entropy
  return entropy

####3. Looking for best splitting variables

In [0]:
#Use dataset partitioning to calculate the function of entropy
def choose_best_split(Work):
  enrtroy_base=calc_entropy(Work)#calculat initial entropy
  num=len(dataset[0])-1 #X
  best_info_gain=0.0;
  best_feature=-1
  for i in range(num):#traversing the variables that each column needs to split
    featlist=[example[i] for example in Work] #value of column X
    unique_values=set(featlist) #values that are not repeated in column X
    entroy_new=0.0#initialization condition entropy
    for value in uniqu_values:#traversing the values that are not repeated in column X
      sub_dataset=splitdata(Work,i,value)#distinguish X column
      prob=len(sub_dataset)/float(len(Work))#calculat probability
      entroy_new+=prob*calc_entropy(sub_dataset)
    info_gain=entroy_base-entrroy_new
    #calculat best splitting variables
    if(info_gain>best_info_gain):
      best_info_gain=info_gain
      best_feature=i
  return best_feature

####4. Defining majority voting method

In [0]:
import operator
def majoritycnt(classlist):
    classcounts={}
    for vote in classlist:
        if vote not in classcounts.keys():
            classcounts[vote]=0
            classcounts[vote]+=1
        sorted_classcount=sorted(classcounts.iteritems(),
                                 key=operator.itemgeeter(1),reverse=True)
        return sorted_classcount[0][0]


####5. Build tree

In [0]:
def create_tree(Work,labels):
  classlist = [example[-1] for example in dataset]#different Y
  #Condition one:All y of dataset is same, return that label
  if classlist. count (classlist [0])==len(classlist):
      return classlist [o] #return this label
  #Coudition two:All label used
  if len(dataset [0])==1:#travesing all
      return majoritycnt (classlist)
  best_feature=choose_best_split(dataset) #find the column where the optimal split is located
  best_feature_label=labels[best_feature] #give label
  my_tree= {best_feature_label:{}}#build new tree
  del(labels[best_feature])
  feature_values= [example [best_feature] for example in dataset] 
  unique_values=set (feature_values) 
  for value in unique_values:
      sublabels =labels [:]
      #build tree
      my_tree[best_feature_label][value]=create_tree(splitdata(dataset,best_feature,value),sublabels)
  return my_tree


####6. Get tree

In [0]:
#Get the number of nodes
def get_leafs_num(mytree):
  numleafs=0
  firststr=list(mytree.keys())[0]
  seconddict=mytree[firststr]
  for key in seconddict.keys():#travesing
    if type(seconddict[key])==dict:
      numleafs+=get_leafs_num(seconddict[key])
    else:
      numleafs+=1
  return numleafs
#Get the number of tree layers
def get_tree_depth (mytree) :
    maxdepth=0
    firststr=list(mytree.keys()) [0]
    seconddict=mytree [firststr ]
    for key in seconddict.keys() :
      if type (seconddict[key])==dict:
          thisdepth+=get_tree_depth (seconddict[key])
      else: 
          thidepth=1
      if thisdepth>maxdepth:
          maxdepth= thisdepth
    return maxdepth
    

####7. Show data

In [0]:
import matplotlib.pyplot as plt
decision_node=dict(boxstyle="sawtooth",fc="0.8")
leaf_node=dict(boxstyle="round4",fc="0.8")
arrow_args=dict (arrowstyle="<-")
def plotnode(nodetext, centerpt, parentpt, nodetype):
  Createplot.ax1.annotate (nodetext,xy=parentpt, xycoords='axes fraction',
                           xytext=centerpt, textcoords='axesfr action',va="center",
                           ha="center",bbox=nodetype, arrowprops=arrow_arge)

def plotmidtext(cntrpt,parentpt,txtstring):
  xmid= (parentpt [0]-cntrpt[0])/2.0+cntrpt [0]
  ymid= (parentpt [1]-cntrpt[1])/2.0+cntrpt [1]
  createplot.a1.tert (xmid.smid.txtstring)
def plottree (mytree, parentpt, nodetxt) :
  numleafs=get_leafs_num (mytree)
  depth=get_tree_depth (mytree)
  firststr=list(mytree.keys0) [0]
  cntrpt=(plottree.xoff+(1.0+float(numleafs))/2/plottree.totalW, plottree.yoff)
  plotmidtext (cntrpt,parentpt,nodetxt)
  plotnode (firststr,cntrpt,parentpt,decisionnode)
  seconddict=mytree [firststr]
  plottree.yoff=plottree.yoff- 1/plottree.totalD
  for key in seconddict.keys ():
      if type (secondict[key])==dict:
          plottree (seconddict [key], cntrpt, str (key))
      else:
          plottree. xoff=plottree. xoff+1.0/plottree. totalW
          plotnode (seconddict [key], (plottree.xoff,plottree.yoff),cntrpt,leaf_node)
          plotmidtext ( (plottree. xoff, plottree. yoff), cntrpt, str (key))
  plottree. yoff=plottree. yoff+1/plottree. totalD




####8. Build image 

In [0]:
def createplot(intree):
    fig=plt.figure(1,facecolor='white')
    fig.clf()
    axprops=dict(xticks=[],yticks=[])
    createplot.ax1=plt.subplot(111,frameon=False,**axprops)
    plottree.totalW=float(get_leafs_num(intree))
    plottree.totalD=float(get_tree_depth(intree))
    plottree.xoff=-0.5/plottree.totalW;
    plottree.yoff=1.0;
    plottree(intree,(0.5,1.0),'')
    plt.show()


####9. Output

In [0]:
createplot(Work)